In [4]:
import sys        
import traceback
import random
import math
import time
from scipy.integrate import *

import numpy as np
%reload_ext Cython


import pyopencl as cl
import pyopencl.cltypes

In [1]:
class Position:
    _x = 0
    _y = 0
    _z = 0
    def __init__ (self,x,y,z):
        self._x = x
        self._y = y
        self._z = z
    def print_info(self):
        print ('_x:',self._x, '_y:',self._y, '_z:',self._z)
        
class Speed:
    _u = 0
    _v = 0
    _w = 0
    def __init__ (self,u,v,w):
        self._u = u
        self._v = v
        self._w = w
    def print_info(self):
        print ('_u:',self._u, '_v:',self._v, '_w:',self._w)

class Item(Position, Speed):
    _m = 0
    _color = ''
    _lifetime = 0    
    def __init__(self,x,y,z,u,v,w,m,color,lifetime):
        Position.__init__(self, x, y, z)
        Speed.__init__(self, u, v, w)
        self._m = m
        self._color = color
        self._lifetime = lifetime
    def print_info(self):
        print ('_x:',self._x, '_y:', self._y, '_z:', self._z,'_u:', self._u, '_v:', self._v, '_w:', self._w, '_m:', self._m, '_color:', 

self._color, '_lifetime', self._lifetime)
        
def items_generate(N): 
    global s1
    color_list = ["green", "red", "blue", "yellow", "orange"]
    for i in range(N):
        x_item = random.uniform(-1, 1)
        y_item = random.uniform(-1, 1)
        z_item = random.uniform(-1, 1)
        u_item = random.uniform(-1, 1)
        v_item = random.uniform(-1, 1)
        w_item = random.uniform(-1, 1)
        m_item = random.uniform(100000, 100000000)
        color_item = random.choice(color_list)
        time_item = random.uniform(1000000, 5000000000)
        s1.append(Item(x_item,y_item,z_item,u_item,v_item,w_item,m_item,color_item,time_item))
#        for i in range(len(s1)):
#            print(s1[i].print_info())    
    return

In [ ]:
def Opencl(s1, num_of_iterations, delta_t):
    Len = len(s1)
    ls = []
    list_of_mass = []
    for m in range(len(s1)):
        ls.append(s1[m]._x)
        ls.append(s1[m]._y)
        ls.append(s1[m]._z)
        ls.append(s1[m]._u)
        ls.append(s1[m]._v)
        ls.append(s1[m]._w)
        list_of_mass.append(s1[m]._m)
    N = np.array(Len)
    Count = np.array(num_of_iterations)
    tau = np.array(delta_t)
    
    ctx = cl.create_some_context()
    queue = cl.CommandQueue(ctx)
    
    mass_array = np.array(list_of_mass,dtype=cl.cltypes.float)
    array_of_coord_and_velocity = np.array(ls, dtype=cl.cltypes.float)
    array_of_coord_and_velocity_new = np.zeros((N, 6), dtype=cl.cltypes.float)
    
    result = np.zeros((Count, N, 6), dtype=cl.cltypes.float)
    a = np.zeros((N, 3), dtype=cl.cltypes.float)
    a_new = np.zeros((N, 3), dtype=cl.cltypes.float)
    
    mf = cl.mem_flags
    buff_list = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=array_of_coord_and_velocity)
    buff_of_list_new = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=array_of_coord_and_velocity_new)
    buff_of_a = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a)
    buff_of_a_new = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=a_new)
    buff_of_mass = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=mass_array)
    buff_of_result = cl.Buffer(ctx, mf.WRITE_ONLY, result.nbytes)
    Count_cl = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=Count)
    N_cl = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=N)
    tau_cl = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=tau)
    
    prg = cl.Program(ctx,
                         """
                         float norm(__global float *array_of_coord_and_velocity, int i, int j)
                         {
                             double temp=0;
                             for (int k=0; k<3; ++k)
                                 temp+=pow(array_of_coord_and_velocity[6*i+k]-array_of_coord_and_velocity[6*j+k],2);
                             return sqrt(temp);
                         }
                         void acceleration(__global float *array_of_coord_and_velocity, __global float *mass_array,__global float *a, int N)
                         {
                             double mass_coef = 5.97 * pow(10.0, 24);
                             double G = 6.67 * pow(10.0,-20)*mass_coef;
                            
                             for (int i=0; i<N; ++i)
                             {
                                 for (int k=0; k<3; ++k)
                                     a[3*i+k]=0;
                                 for (int j=0; j<N; ++j)
                                     if (j!=i)
                                         for (int k=0; k<3; ++k)
                                             a[3*i+k]+=G*mass_array[j]*(array_of_coord_and_velocity[6*j+k]-array_of_coord_and_velocity[6*i+k])/pow(norm(array_of_coord_and_velocity,i,j),3);
                             }
                         }
                         __kernel void opencl_cl(__global float *array_of_coord_and_velocity, __global float *mass_array, __global float *result, __global double *tau_cl, __global int *Count_cl, __global int *N_cl, __global float *a, __global float *a_new, __global float *array_of_coord_and_velocity_new)
                         {   
                             int Count=*Count_cl;
                             int N=*N_cl;            
                             
                             double tau=*tau_cl;
                             double mass_coef = 5.97 * pow(10.0, 24);
                             double G = 6.67 * pow(10.0,-20)*mass_coef;
                            
                             for (int i=0; i<N; ++i)
                             {
                                 for (int k=0; k<3; ++k)
                                     a[3*i+k]=0;
                                 for (int j=0; j<N; ++j)
                                     if (j!=i)
                                         for (int k=0; k<3; ++k)
                                             a[3*i+k]+=G*mass_array[j]*(array_of_coord_and_velocity[6*j+k]-array_of_coord_and_velocity[6*i+k])/pow(norm(array_of_coord_and_velocity,i,j),3);
                             }
                             
  
                             for (int j=0; j<N; ++j)
                                 for (int k=0; k<6; ++k)
                                     result[6*j+k]=array_of_coord_and_velocity[6*j+k];
                                     
                             for (int i=1; i<Count; ++i)
                             {
                                 for (int j=0; j<N; ++j)
                                     for (int k=0; k<3; ++k)
                                         {
                                            array_of_coord_and_velocity_new[6*j+k]=array_of_coord_and_velocity[6*j+k]+array_of_coord_and_velocity[6*j+k+3]*tau+0.5*a[3*j+k]*tau*tau;
                                         }
                                
                                         
                                 acceleration(array_of_coord_and_velocity_new,mass_array,a_new,N);
                                 for (int j=0; j<N; ++j)
                                     for (int k=0; k<3; ++k)
                                         array_of_coord_and_velocity_new[6*j+k+3]=array_of_coord_and_velocity[6*j+k+3]+0.5*(a[3*j+k]+a_new[3*j+k])*tau;
                                 for (int j=0; j<N; ++j)
                                     for (int k=0; k<3; ++k)
                                     {
                                         array_of_coord_and_velocity[6*j+k]=array_of_coord_and_velocity_new[6*j+k];
                                         array_of_coord_and_velocity_new[6*j+k]=0;
                                         array_of_coord_and_velocity[6*j+k+3]=array_of_coord_and_velocity_new[6*j+k+3];
                                         array_of_coord_and_velocity_new[6*j+k+3]=0;
                                         a[3*j+k]=a_new[3*j+k];
                                         result[6*N*i+6*j+k]=array_of_coord_and_velocity[6*j+k];
                                         result[6*N*i+6*j+k+3]=array_of_coord_and_velocity[6*j+k+3];
                                     }
                             }
                         }""") 
    try:
        prg.build()
    except:
        print("Error:")
        print(prg.get_build_info(ctx.devices[0], cl.program_build_info.LOG))
        raise
    start_time = time.time()
    prg.opencl_cl(queue, (1,), None, buff_list, buff_of_mass, buff_of_result,tau_cl, Count_cl, N_cl, buff_of_a, buff_of_a_new, buff_of_list_new)
    cl.enqueue_read_buffer(queue, buff_of_result, result).wait()
    cl.enqueue_read_buffer(queue, buff_of_a, a).wait()    
    return time.time() - start_time

s1 = []
items_generate(100)
print("OpenCL_Verlet: ", Opencl(s1[:], 5, 1))

s1 = []
items_generate(200)
print("OpenCL_Verlet: ", Opencl(s1[:], 5, 1))

s1 = []
items_generate(400)
print("OpenCL_Verlet: ", Opencl(s1[:], 5, 1))

s1 = []
items_generate(1000)
print("OpenCL_Verlet: ", Opencl(s1[:], 5, 1))
    